# Transformerの詳細

- この章では、Transformerの詳細な実装について記載されている。
- 実際にPyTorchを使って実装を進めていく

## 3.1 Transformerのアーキテクチャ

- オリジナルのTransformerはエンコーダ・デコーダアーキテクチャとなっている。
- もともと機械翻訳向けにこの構造をしている。

<img src="img/ml-transformers-chap03-transformer-anatomy_2022-08-25-22-02-10.png" />

- エンコーダの特徴
  - 入力トークン系列を埋め込みベクトルの系列に変換する。
  - 埋め込みベクトルは、隠れ状態やコンテキストとも呼ばれる。
- デコーダの特徴
  - 埋め込みベクトルの系列を入力し、出力トークン系列を生成する。
  - デコーダの終了は、特別なEOSトークンに到達するまで継続される。

- その後エンコーダ・デコーダのそれぞれが独立したモデルとして適応されていくこととなる。

- エンコーダのみモデル
  - テキスト分類や固有表現認識といったタスクに使用される。
  - このアーキテクチャでは、与えられたあるトークンの結果が、前後双方のコンテキストに依存する。
  - これは双方向アテンションと呼ばれ、BERT系が該当する。
- デコーダのみモデル
  - 次の単語を文脈から予測するようなタスクに使用される。
  - このアーキテクチャでは、与えられたあるトークンの結果が、前方のコンテキストのみに依存する。
  - これは因果的もしくは自己回帰型アテンションと呼ばれ、GPT系が該当する。
- エンコーダ・デコーダモデル
  - 機械翻訳や要約といったタスクに使用される。
  - BARTやT5がこのアーキテクチャに該当する。

- 実際にはこれらの区別はあいまいであるので注意が必要。

## 3.2 エンコーダ

- Transformerのエンコーダは複数のエンコーダをスタックする。
- エンコーダは文脈情報を埋め込んだ表現を生成する。
- エンコーダは、マルチヘッドセルフアテンションと単純な順伝搬層で構成される。

<img src="img/ml-transformers-chap03-transformer-anatomy_2022-08-25-22-14-41.png" />

### 3.2.1 セルフアテンション

- アテンションは系列の各要素に異なる重みを割り当てる機構である。
- テキストの場合、系列の要素はトークン埋め込みである。
- トークン埋め込みは、固定次元のベクトルで表される。
  - 例えばBERTの場合、768次元のベクトルとなる。
- セルフアテンションは、入力されるトークン埋め込み$x_j$すべてを使用した線形和を、その系列ぶん計算する。

$$ x^{'}_i = \sum^n_{j=1}{w_{ji}x_j}$$

- $w_{ji}$はアテンションの重みと呼ばれ、$\sum_{j}{w_{ji}}=1$となるように正規化される。
- セルフアテンションは以下のように、同一単語であっても周囲の文脈を考慮した埋め込みを生成することができる。

<img src="img/ml-transformers-chap03-transformer-anatomy_2022-08-26-09-09-37.png" />

- この出力される結果を文脈埋め込みと呼び、Transformerに選考してELMoなどの言語モデルで取り入れられている。
  - [Deep Contextualized Word Representations (2018-02-15)](https://arxiv.org/abs/1802.05365)
- 以降は$w_{ji}$を計算する方法について述べる。

#### 3.2.1.1 スケール化ドット積アテンション

- いくつか実装があるがTransformerでは以下のステップを踏む。
  - 各トークン埋め込みをクエリ・キー・バリューというベクトルに射影する。
  - クエリとキーの類似度を計算し、これをアテンションスコアと呼ぶ。
    - アテンションスコアは、系列長をnとすると、n x n行列となる。
  - アテンションスコアは任意の数を生成できるため、softmaxで正規化して、これを重み$ w_{ji} $とする。
    - 正規化は入力系列方向に行い、$ \sum_{j}{w_{ji}}=1 $となるようにする
  - バリューベクトル$ v_1,...,v_n $の重み付け線形和$ x^{'}_i = \sum^n_{j=1}{w_{ji}v_j} $で文脈埋め込みを計算する
- これらの可視化をJupyter向けのBertVizにおけるneuron_viewで行うことができる。


In [ ]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)